In this short tutorial we will install and use [MARTINI](https://kyleaoman.github.io/martini/build/html/includeme.html), an analysis package for creating mock HI-data cubes similar to radio interferometer data, written by Kyle Oman (kyle.a.oman@durham.ac.uk). This example uses input from the [Simba](https://ui.adsabs.harvard.edu/abs/2019MNRAS.486.2827D/abstract) simulations. For data access, enquire with [Romeel Davé](https://www.ph.ed.ac.uk/people/romeel-dave).

![MARTINI](http://kyleaoman.github.io/martini/build/html/_images/martini_banner.png)

MARTINI is a modular package for the creation of synthetic resolved HI line observations (data cubes) of smoothed-particle hydrodynamics simulations of galaxies. The various aspects of the mock-observing process are divided logically into sub-modules handling the data cube, source, beam, noise, spectral model and SPH kernel. MARTINI is object-oriented: each sub-module provides a class (or classes) which can be configured as desired. For most sub-modules, base classes are provided to allow for straightforward customization. Instances of each sub-module class are then given as parameters to the Martini class. A mock observation is then constructed by calling a handful of functions to execute the desired steps in the mock-observing process.

## Installation

Note that martini requires `python3`.

We will use `git` and `pip` to download and install [MARTINI from github](https://github.com/kyleaoman/martini). To do so:

1. open a terminal and navigate to a directory where the source code will be stored
2. type the following commands:

```
git clone https://github.com/kyleaoman/martini
python3 -m pip install --user -e martini/[simbasource,hdf5_output]
```

If you have superuser permissions or use a virtual environment, you may wish to remove the --user flag. If you do not want support for `.hdf5` format output, you may remove `,hdf5_output` (output in `.fits` format is supported by default). With these commands required dependencies will be fetched and installed automatically. Check for error messages during installation. For greater control you may also install the dependencies by hand. These are: numpy >= 1.15.3, astropy >= 3.0, scipy and h5py.

You may check for basic functionality of martini by running:

In [ ]:
from martini import demo

demo()

If this produces errors, you may need to restart the Python kernel of this notebook so that it sees the recently installed packages (Kernel -> Restart in the menubar).

When run successfully, this will make a mock observation of a very simple analytic disc model and write some output to the working directory.

## Simba Data

This example uses data from the Simba simulations. These data are not currently publicly available, but the authors of the simulations are reasonably open to collaboration. Enquire with [R. Davé](https://www.ph.ed.ac.uk/people/romeel-dave) for further information.

In this example, we will use a snapshot file `snap_m25n512_151.hdf5` and the corresponding galaxy catalogue file `m25n512_151.hdf5`. Other snapshots can be substituted for this one, simply substitute the filenames below.

## Simba Example

First, import some modules from Martini, and the units module from astropy.

In [ ]:
from martini.sources import SimbaSource
from martini import DataCube, Martini
from martini.beams import GaussianBeam
from martini.noise import GaussianNoise
from martini.spectral_models import GaussianSpectrum
from martini.sph_kernels import (
    AdaptiveKernel,
    CubicSplineKernel,
    GaussianKernel,
    DiracDeltaKernel,
)
import astropy.units as U

The different martini sub-modules need to be initialized, reading [this overview](https://kyleaoman.github.io/martini/build/html/martini.html) is recommended before continuing this example. 

See the [full documentation](https://kyleaoman.github.io/martini/build/html/) for the individual sub-modules for details of all configuration options. A few suggested best-practices specific to Simba are outlined below.

### SOURCE
Edit the call to `SimbaSource` below to point to the snapshot and group files. The `snapPath` argument should give the directory
containing the snapshot file, and the `snapName` argument should
give the name of the snapshot file. Similarly, the `groupPath` and `groupName` arguments should give the directory containing and name of the group catalogue file, respectively.

The `groupID` specifies the identifier of the object to use as a source, and is matched against the dataset of the same name in the group catalogue. For this example, we will use `groupID=15`, which is a galaxy with a massive neutral gas disc in snapshot `151` of simulation `m25n512`. If using another snapshot as input, you should use the group catalogue to identify an object of interest.

The argument `aperture` controls the radial extent of a region
to load around the object of interest, in physical (not comoving,
no little h) units. Using larger values will include more 
foreground/background, which may be desirable, but will also slow 
down execution and can impair the automatic routine used to find a 
disc plane. Normally it is advisable to set this to approximately
the virial radius of the source object, or just large enough to
capture the region of interest around it (e.g. enough to encompass
the host of a satellite galaxy).

The rotation configuration takes an inclination (here 60deg) and
rotation about the pole (here 90deg). The code attempts to
automatically align the galactic disk in the y-z plane by aligning
the angular momentum along the x-axis. The polar rotation is then
applied, and finally the disc inclined by a rotation around the
y-axis (the line of sight is along the x-axis). The automatic
alignment will work for typical reasonably isolated discs, but will
struggle when companions are present, when the angular momentum axis
is a poor tracer of the disc plane, and especially for satellites. If
finer control of the orientation is needed, derive the transformation
from the simulation box coordinates to the desired coordinates for
the 'observation', keeping in mind that the line of sight is along
the x-axis. This rotation matrix can then be passed to rotation as
{'rotmat': np.eye(3)} (here the identity rotation matrix used as an
example). A common problem in this case is deriving the inverse
transform instead of the forward transform, if unexpected results are
obtained, first try passing the transpose of the rotation matrix.

In [ ]:
simlabel = "m25n512"
snap = 151
groupID = 15

source = SimbaSource(
    snapPath="./",
    snapName="snap_{:s}_{:03d}.hdf5".format(simlabel, snap),
    groupPath="./",
    groupName="{:s}_{:03d}.hdf5".format(simlabel, snap),
    groupID=groupID,
    aperture=100.0 * U.kpc,
    distance=4.0 * U.Mpc,
    vpeculiar=0 * U.km / U.s,
    rotation={"L_coords": (60 * U.deg, 90.0 * U.deg)},
    ra=0.0 * U.deg,
    dec=0.0 * U.deg,
)

### DATACUBE
It is usually advisable to set the centre of the cube to track the
centre of the source, as illustrated below. Note that the source
systemic velocity is set according to the distance, peculiar velocity, and Hubble's law.
These values can instead be set explicitly, if desired. A datacube
with 128x128 pixels usually takes a few minutes, depending on the number of particles. 1024x1024 can easily take
several hours. The number of channels has less influence on the
runtime. Most of the runtime is spent when `M.insert_source_in_cube` is
called below.

In [ ]:
datacube = DataCube(
    n_px_x=384,
    n_px_y=384,
    n_channels=50,
    px_size=10.0 * U.arcsec,
    channel_width=16.0 * U.km * U.s**-1,
    velocity_centre=source.vsys,
    ra=source.ra,
    dec=source.dec,
)

### BEAM
It is usually advisable to set the beam size to be ~3x the pixel
size. Note that the data cube is padded according to the size of the
beam, this usually results in the number of pixel rows printed in the
progress messages to differ from the requested dimensions. The
padding is required for accurate convolution with the beam, but
contains incorrect values after convolution and is discarded to
produce the final data cube of the requested size.

In [ ]:
beam = GaussianBeam(
    bmaj=30.0 * U.arcsec, bmin=30.0 * U.arcsec, bpa=0.0 * U.deg, truncate=3.0
)

### NOISE
The noise is normally added before convolution with the beam (as
below in this example). The rms value passed is for the noise before
convolution, the rms noise in the output data cube will therefore
typically differ from this value.

In [ ]:
noise = GaussianNoise(rms=1.0e-7 * U.Jy * U.arcsec**-2)

### SPECTRAL MODEL
The 'subgrid' velocity dispersion can also be fixed to a constant
value, e.g. `sigma=7 * U.km / U.s`.

In [ ]:
spectral_model = GaussianSpectrum(sigma="thermal")

### SPH KERNEL
The Simba simulation uses a Cubic Spline smoothing kernel. The implementation of
this kernel included in Martini uses an approximation
which breaks down when the particle smoothing lengths are small
compared to the size of the pixels (at the distance of the source).
If you obtain an error complaining about the smoothing length relative to the pixel scale,
you may either disable this check following the instructions in the error message (which will result in some inaccuracy), or try a different kernel - the same FWHM will be used automatically - which may be more tolerant (which results in a different sort of inaccuracy). In situations where most smoothing lengths are compatible with the desired kernel, using the AdaptiveKernel module may be desirable: this accepts a list of kernels in order of decreasing priority and uses the first which is sufficiently accurate on a per-particle basis. This is illustrated in the example below.

If you choose to disable the check on the kernel integration accuracy, note that this can result in significant missing or extra mass. A suggested check would be to make a mock observation at a resolution where no error is produced and compare the integrated HI mass with the mock observation at the desired resolution. If these differ substantially, do not proceed with science! If they do not differ substantially, it *might* be ok to proceed, with caution.

In [ ]:
sph_kernel = AdaptiveKernel((CubicSplineKernel(), GaussianKernel(truncate=6)))

Now set up the configuration, and do the actual run:

In [ ]:
M = Martini(
    source=source,
    datacube=datacube,
    beam=beam,
    noise=noise,
    spectral_model=spectral_model,
    sph_kernel=sph_kernel,
)

### Run the calculation

Progress messages will be printed every `printfreq` rows; suppress by setting to `None`.

In [ ]:
M.insert_source_in_cube(printfreq=50)
M.add_noise()
M.convolve_beam()

You may notice that the number of pixels in the progress counter differs from the number defined in the DataCube module. This is because convolution with the beam requires some padding, which is ultimately filled with nonsense and discarded.

To write the results: two output formats are available, depending on preference. Both
formats are self-documenting, via FITS header keywords and HDF5
attributes, respectively. For HDF5 output, the beam image is included
in the same file. (If you do not have hdf5, comment out the call to `write_hdf5`.)

In [ ]:
M.write_fits("simba_martini_demo.fits", channels="velocity")
M.write_beam_fits("simba_martini_demo_beam.fits", channels="velocity")
M.write_hdf5("simba_martini_demo.hdf5", channels="velocity")

### Inspect the results

Let's load the HDF5 that MARTINI produced and take a quick look.

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS

with fits.open("simba_martini_demo.fits") as f:
    FluxCube = f[0].data
    Nchannels = FluxCube.shape[1]
    vch = np.array(
        WCS(f[0].header).sub(axes=[3]).all_pix2world(np.arange(Nchannels), 0)
    )[0]
    vch = (vch * U.m / U.s - source.vsys).to(U.km / U.s).value

In [ ]:
FluxCube.shape
FluxCube = FluxCube[0]  # strip the Stokes' axis
FluxCube.shape

Let's examine one of the velocity channels:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(5, 5))
plt.clf()
ax = fig.add_subplot(1, 1, 1)

plt.imshow(FluxCube[27, :, :], cmap="Greys", aspect=1.0, origin="lower")
ax.autoscale(False)
ax.set_xlabel("x [px = arcsec/{:.0f}]".format(datacube.px_size.to(U.arcsec).value))
ax.set_ylabel("y [px = arcsec/{:.0f}]".format(datacube.px_size.to(U.arcsec).value))
plt.colorbar(label="Flux [Jy/beam]");

And do a quick plot of the first three moments:

In [ ]:
import numpy as np

rms = np.std(
    FluxCube[:, :20, :20]
)  # noise in a corner patch where there is little signal
clip = np.where(FluxCube > 5 * rms, 1, 0)
np.seterr(all="ignore")
fig = plt.figure(figsize=(16, 5))
plt.clf()
sp1 = fig.add_subplot(1, 3, 1)
sp2 = fig.add_subplot(1, 3, 2)
sp3 = fig.add_subplot(1, 3, 3)
mom0 = np.sum(FluxCube, axis=0)
mask = np.where(mom0 > 0.002, 1, np.nan)
mom1 = np.sum(FluxCube * clip * vch[:, np.newaxis, np.newaxis], axis=0) / mom0
mom2 = np.sqrt(
    np.sum(
        FluxCube
        * clip
        * np.power(vch[:, np.newaxis, np.newaxis] - mom1[np.newaxis], 2),
        axis=0,
    )
    / mom0
)
im1 = sp1.imshow(mom0, cmap="Greys", aspect=1.0, origin="lower")
plt.colorbar(im1, ax=sp1, label="mom0 [Jy/beam]")
im2 = sp2.imshow(
    (mom1 * mask), cmap="RdBu", aspect=1.0, origin="lower", vmin=-250, vmax=250
)
plt.colorbar(im2, ax=sp2, label="mom1 [km/s]")
im3 = sp3.imshow(
    (mom2 * mask), cmap="magma", aspect=1.0, origin="lower", vmin=0, vmax=50
)
plt.colorbar(im3, ax=sp3, label="mom2 [km/s]")
for sp in sp1, sp2, sp3:
    sp.set_xlabel("x [px = arcsec/{:.0f}]".format(datacube.px_size.to(U.arcsec).value))
    sp.set_ylabel("y [px = arcsec/{:.0f}]".format(datacube.px_size.to(U.arcsec).value))
plt.subplots_adjust(wspace=0.3)

This galaxy clearly has a very nice spiral morphology in HI, slightly lopsided about the potential minimum (where the image is centred) and a nice rotation dominated velocity field. The alignment of the disc looks as expected - the inclination looks to be about 60 degrees, and the position angle is horizontal in the figure - in this case the automated orientation function has performed well, though it should never be assumed that this will always be the case!

For complete documentation, more usage examples, and further information, please take a look at the [MARTINI webpage](https://kyleaoman.github.io/martini).

If for some reason the figures above fail to appear, you may try uncommenting and running:

In [ ]:
# plt.show()